<a href="https://colab.research.google.com/github/Geeteshkamble/AI/blob/main/nb/Gemma3_(1B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

Load up `Gemma 3 1B Instruct`, and set parameters

In [3]:
from unsloth import FastModel
import torch
max_seq_length = 1024

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
Unsloth: Successfully patched SmolVLMForConditionalGeneration for better torch.compile compatibility.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-29 11:58:10 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 04-29 11:58:10 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.4.1: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.5.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using 

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [4]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [5]:
import pandas as pd
import numpy as np
from datasets import Dataset

In [6]:
# Load the CSV file
df = pd.read_csv('/content/Negative words.csv')

In [7]:
df.columns

Index(['Child ', 'Robot '], dtype='object')

In [21]:
df.columns = ['Child ', 'Robot ']

In [9]:
# Create a system prompt for empathetic responses
system_prompt = """You are a friendly and helpful robot assistant for children. When a child expresses negative feelings or statements,
respond with understanding, positivity, and encouragement. Always maintain a supportive and caring tone.
Use simple language that children can understand and provide constructive, positive responses."""

In [22]:
# Prepare the dataset
dataset = []
for idx, row in df.iterrows():
    dataset.append({
        "prompt": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": str(row['Child '])}, # Convert to string explicitly
        ],
        "answer": str(row['Robot ']) # Convert to string explicitly
    })

# Convert to HuggingFace dataset
dataset = Dataset.from_list(dataset)

In [23]:
# Split into train and test sets
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset['train']
test_dataset = dataset['test']

In [25]:
print("Example training sample:")
print(train_dataset[0])

Example training sample:
{'prompt': [{'content': 'You are a helpful and empathetic assistant. When someone expresses negative feelings or statements, \nrespond with understanding, positivity, and encouragement. Always maintain a supportive and caring tone.', 'role': 'system'}, {'content': 'I don’t want to do this assignment!', 'role': 'user'}], 'answer': 'We’ll work on it little by little, and it will get done!'}


In [16]:
def evaluate_response(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]

        # Check for positive language
        positive_words = ['great', 'fun', 'happy', 'help', 'together', 'awesome', 'wonderful', 'exciting']
        for word in positive_words:
            if word in response.lower():
                score += 0.5

        # Check for empathetic language
        empathetic_phrases = ['I understand', 'that\'s okay', 'let\'s try', 'we can', 'together']
        for phrase in empathetic_phrases:
            if phrase.lower() in response.lower():
                score += 1.0

        # Penalize negative language
        negative_words = ['no', 'can\'t', 'won\'t', 'don\'t', 'bad', 'wrong']
        for word in negative_words:
            if word in response.lower():
                score -= 0.5

        scores.append(score)
    return scores

In [17]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1,
    num_generations = 4,
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    max_steps = 50,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none",
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [18]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [evaluate_response],
    args = training_args,
    train_dataset = train_dataset,
)
trainer.train()

Unsloth: Switching to float32 training since model cannot work with float16


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 424 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 6,522,880/1,006,408,832 (0.65% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / evaluate_response
1,-0.000000,0.625000,0.478714,109.750000,0.000000,0.625000
2,-0.000000,0.500000,0.912871,146.500000,0.000000,0.500000
3,0.000000,-0.125000,0.478714,149.000000,0.000023,-0.125000
4,0.000000,0.750000,0.500000,170.500000,0.000019,0.750000
5,0.000000,0.000000,1.154701,208.000000,0.000023,0.000000
6,0.000000,0.250000,0.500000,186.500000,0.000037,0.250000
7,0.000000,1.250000,1.190238,268.250000,0.000024,1.250000
8,0.000000,0.750000,0.866025,168.000000,0.000051,0.750000
9,0.000000,0.125000,0.629153,157.000000,0.000088,0.125000
10,0.000000,0.375000,1.108678,185.500000,0.000106,0.375000


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=50, training_loss=9.702265706437175e-06, metrics={'train_runtime': 1114.8596, 'train_samples_per_second': 0.179, 'train_steps_per_second': 0.045, 'total_flos': 0.0, 'train_loss': 9.702265706437175e-06})

In [19]:
test_cases = [
    # "I don't want to do my homework!",
     "I'm scared of the dark!",
    # "I don't want to share my toys!",
    # "I don't like my teacher!",
    # "I'm not good at this game!"
]


In [20]:
for test_case in test_cases:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": test_case},
    ]

    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,
        tokenize = False,
    )

    print(f"\nTest case: {test_case}")
    print("Model response:")
    from transformers import TextStreamer
    _ = model.generate(
        **tokenizer(text, return_tensors = "pt").to("cuda"),
        max_new_tokens = 128,
        temperature = 0.7,
        top_p = 0.9,
        top_k = 50,
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )



Test case: I'm scared of the dark!
Model response:
<bos><bos><start_of_turn>user
You are a helpful and empathetic assistant. When someone expresses negative feelings or statements, 
respond with understanding, positivity, and encouragement. Always maintain a supportive and caring tone.

I'm scared of the dark!<end_of_turn>
<start_of_turn>model
Oh, sweetheart, that’s really brave of you to share that with me. It’s completely understandable that you’re feeling scared of the dark. It’s a really common fear, and it’s okay to feel that way.  

It sounds like you’re experiencing a lot of anxiety right now, and that’s a really tough feeling.  Just recognizing that you’re scared is a huge first step.  

Let’s just take a deep breath together.  It’s okay to feel scared.  There’s nothing wrong with feeling this way.  

Sometimes, when we’
